# ATC predict by morning & afternoon prices - Analytics

### Import Library

In [12]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [13]:
import os
from pathlib import Path
notebook_path = os.getcwd()
current_dir = Path(notebook_path)
csv_file = str(current_dir) + '/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [14]:
data = dataset.copy()

In [15]:
data = data[data.index > '2020-11-01 00:00:00']

In [16]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2025-02-14 14:15:00,1343.0,1343.0,1340.3,1341.3,7141
2025-02-14 14:20:00,1340.9,1341.9,1340.5,1341.4,4593
2025-02-14 14:25:00,1341.1,1342.5,1340.7,1342.5,4207


## Intraday(Morning + Affternoon) data 

In [17]:
def get_data_11h25(tick):
  tick = tick[100*tick.index.hour+tick.index.minute == 1125]
  if not tick.empty:
    return tick[0]
      
def get_data_13h00(tick):
  tick = tick[100*tick.index.hour+tick.index.minute == 1300]
  if not tick.empty:
    return tick[0]

def get_data_14h25(tick):
  tick = tick[100*tick.index.hour+tick.index.minute == 1425]
  if not tick.empty:
    return tick[0]

def get_max_price_morning(tick):
  tick = tick[100*tick.index.hour+tick.index.minute >= 900]
  tick = tick[100*tick.index.hour+tick.index.minute <= 1130]
  return tick.max()

def get_min_price_morning(tick):
  tick = tick[100*tick.index.hour+tick.index.minute >= 900]
  tick = tick[100*tick.index.hour+tick.index.minute <= 1130]
  return tick.min()

def get_max_price_affternoon(tick):
  tick = tick[100*tick.index.hour+tick.index.minute >= 1300]
  tick = tick[100*tick.index.hour+tick.index.minute <= 1425]
  return tick.max()

def get_min_price_affternoon(tick):
  tick = tick[100*tick.index.hour+tick.index.minute >= 1300]
  tick = tick[100*tick.index.hour+tick.index.minute <= 1425]
  return tick.min()

In [18]:
day_data = data.copy()
day_data['close_11h25'] = day_data['Close']
day_data['open_13h00'] = day_data['Open']
day_data['close_14h25'] = day_data['Close']
day_data['morning_high'] = day_data['High']
day_data['morning_low'] = day_data['Low']
day_data['affternoon_high'] = day_data['High']
day_data['affternoon_low'] = day_data['Low']
day_data['RSI'] = ta.rsi(day_data["Close"], length=14)
day_data['RSI_9h00'] = day_data['RSI']
day_data['RSI_11h25'] = day_data['RSI']
day_data['RSI_13h00'] = day_data['RSI']
day_data['RSI_14h25'] = day_data['RSI']
day_data = day_data.resample('D').agg({
        'Open': 'first',
        'Close': 'last',
        'morning_high': get_max_price_morning,
        'morning_low': get_min_price_morning,
        'affternoon_high': get_max_price_affternoon,
        'affternoon_low': get_min_price_affternoon,
        'close_11h25': get_data_11h25,
        'open_13h00': get_data_13h00,
        'close_14h25': get_data_14h25,
        'RSI_9h00': 'first',
        'RSI_11h25': get_data_11h25,
        'RSI_13h00': get_data_13h00,
        'RSI_14h25': get_data_14h25
    })
day_data.dropna(subset=['Close'], inplace=True)

In [19]:
day_data['morning_ibs'] = day_data.apply(lambda x: (0 if (x["morning_high"] == x["morning_low"]) else (x["close_11h25"] - x["morning_low"]) / (x["morning_high"] - x["morning_low"])), axis=1)
day_data['morning_ibs_x100'] = round(day_data['morning_ibs'] * 100, 1)
day_data['afternoon_ibs'] = day_data.apply(lambda x: (0 if (x["affternoon_high"] == x["affternoon_low"]) else (x["close_14h25"] - x["affternoon_low"]) / (x["affternoon_high"] - x["affternoon_low"])), axis=1)
day_data['afternoon_ibs_x100'] = round(day_data['afternoon_ibs'] * 100, 1)

In [20]:
day_data['RSI_11h25'] = round(day_data['RSI_11h25'], 1)
day_data['RSI_9h00'] = round(day_data['RSI_9h00'], 1)
day_data['RSI_13h00'] = round(day_data['RSI_13h00'], 1)
day_data['RSI_14h25'] = round(day_data['RSI_14h25'], 1)

In [21]:
day_data['atc_return'] = day_data['Close'] - day_data['close_14h25']

In [22]:
day_data

,Open,Close,morning_high,morning_low,affternoon_high,affternoon_low,close_11h25,open_13h00,close_14h25,RSI_9h00,RSI_11h25,RSI_13h00,RSI_14h25,morning_ibs,morning_ibs_x100,afternoon_ibs,afternoon_ibs_x100,atc_return
Date,,,,,,,,,,,,,,,,,,
2020-11-02,900.1,904.0,901.5,895.4,907.0,896.3,898.4,898.0,907.0,51.0,48.2,49.1,68.2,0.491803,49.2,1.000000,100.0,-3.0
2020-11-03,909.3,908.3,910.8,906.6,910.4,903.7,908.0,908.0,908.5,66.7,51.5,47.8,56.3,0.333333,33.3,0.716418,71.6,-0.2
2020-11-04,909.0,912.3,914.6,906.7,918.1,910.2,912.5,912.5,913.5,57.1,56.5,57.4,49.9,0.734177,73.4,0.417722,41.8,-1.2
2020-11-05,913.7,910.2,919.5,912.4,919.9,910.0,918.5,918.5,910.8,50.5,64.7,67.3,32.0,0.859155,85.9,0.080808,8.1,-0.6
2020-11-06,911.2,908.3,912.0,905.0,912.7,903.3,905.3,905.3,910.0,39.3,22.4,29.7,56.6,0.042857,4.3,0.712766,71.3,-1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-10,1333.4,1330.5,1335.0,1325.2,1337.6,1330.0,1333.1,1332.9,1330.0,24.2,49.4,49.9,36.0,0.806122,80.6,0.000000,0.0,0.5
2025-02-11,1332.5,1334.1,1334.0,1327.6,1333.0,1328.9,1330.8,1330.8,1332.4,49.9,50.7,56.3,57.1,0.500000,50.0,0.853659,85.4,1.7
2025-02-12,1337.4,1329.5,1342.0,1337.4,1340.7,1332.8,1339.7,1339.8,1332.8,79.0,59.0,52.2,20.4,0.500000,50.0,0.000000,0.0,-3.3
